In [27]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import numpy as np
from numpy import dot
from numpy.linalg import norm
import jaro
from sklearn.feature_extraction.text import CountVectorizer

def cos_sim(A):
  return dot(A[0], A[1])/(norm(A[0])*norm(A[1]))

def test_filter(name, site1, raw_one1, site2, raw_one2):

    if site1 != 'NTIS' and site2 != 'NTIS':
        author_inst1 = raw_one1['author_inst'].split(";")[raw_one1['author'].split(";").index(name)]
        author_inst2 = raw_one2['author_inst'].split(";")[raw_one2['author'].split(";").index(name)]

        inst = jaro.jaro_winkler_metric(author_inst1, author_inst2)
        weight = 0

        if raw_one1['title'] == raw_one2['title']: #or inst >= 0.8:
            weight = 6 #6
        else:
            journal1 = raw_one1['journal']
            conference1 = raw_one1['issue_inst']
            issYear1 = int(raw_one1['issue_year'][:4])
            coauthor1 = raw_one1['author'].split(";")[1:-1]
            keyword1 = raw_one1['paper_keyword'].replace(" ", "").split(".")

            journal2 = raw_one2['journal']
            conference2 = raw_one2['issue_inst']
            issYear2 = int(raw_one2['issue_year'][:4])
            coauthor2 = raw_one2['author'].split(";")[1:-1]
            keyword2 = raw_one2['paper_keyword'].replace(" ", "").split(".")
            
            joc = 1 if journal1 == journal2 and conference1 == conference2 else -1
            yop = -(2*(abs(issYear1-issYear2)/10)-1)
            co_authorship = 1 - np.exp(-len([i for i in coauthor1 if i in coauthor2]))
            keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

            title1 = raw_one1['title']
            title2 = raw_one2['title']

            title_corpus = [title1, title2]
            title_tfidfv = TfidfVectorizer().fit(title_corpus)
            title_cossim = cos_sim(title_tfidfv.transform(title_corpus).toarray())

            abstract1 = raw_one1['abstract']
            abstract2 = raw_one2['abstract']
            
            if abstract1 == 'null' or abstract2 == 'null':
                abstract_cossim = 0
            else:
                abstract_corpus = [abstract1, abstract2]
                abstract_tfidfv = TfidfVectorizer().fit(abstract_corpus)
                abstract_cossim = cos_sim(abstract_tfidfv.transform(abstract_corpus).toarray())
                print(abstract_corpus)
                print(abstract_tfidfv.transform(abstract_corpus).toarray())

            print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword} | title: {title_cossim} | abstract: {abstract_cossim}')

            weight += joc + yop + co_authorship + keyword + title_cossim + abstract_cossim
    else:
        weight = 0
        print('NTIS는 개발중')

    return weight

In [28]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')

name = '최도진'
site1 = 'Scienceon'
site2 = 'Scienceon'
input_paper1 = ObjectId('61939443e9b04a9d64abe055')
input_paper2 = ObjectId('61939447e9b04a9d64abe06a')

if site1 == 'Scienceon' :
    site_raw1 = client['SCIENCEON']['Rawdata']
elif site1 == 'NTIS' :
    site_raw1 = client['NTIS']['Rawdata']
elif site1 == 'DBPIA' :
    site_raw1 = client['DBPIA']['Rawdata']

if site2 == 'Scienceon' :
    site_raw2 = client['SCIENCEON']['Rawdata']
elif site2 == 'NTIS' :
    site_raw2 = client['NTIS']['Rawdata']
elif site2 == 'DBPIA' :
    site_raw2 = client['DBPIA']['Rawdata']

input_raw1 = site_raw1.find_one({ '_id' : input_paper1 })
input_raw2 = site_raw2.find_one({ '_id' : input_paper2 })

test_filter(name, site1, input_raw1, site2, input_raw2)


온라인 쇼핑몰의 대중화로 인해 소비자는 폭 넓은 소비의 기회를 제공받고 있다. 소비자들은 온라인 쇼핑몰에서 제공되는 실제 상품을 구매한 사용자의 리뷰, 상품의 자세한 정보와 같은 정보를 활용하여 상품의 구매 여부를 결정한다. 방대한 정보가 신뢰할 수 있는 정보인지는 소비자들이 스스로 판단해야만하기 때문에 객관적이며 신뢰할 수 있는 정보의 제공이 필요하다. 본 논문에서는 신뢰할 수 있는 상품의 추천을 위해서 상품의 신뢰도를 고려한 개인화된 추천 기법을 제안한다. 제안하는 추천 기법은 개인화 추천을 위해서 사용자의 다양한 행위를 기반으로 사용자의 선호도를 판별한다. 또한, 최신의 성향을 반영하기 위해서 시간 가중치를 고려한 사용자 선호도 계산 방법을 제안한다. 마지막으로 사용자가 사용하지 않았던 상품에 대한 선호 점수를 예측하고 예측 점수가 높은 상품 중 신뢰도가 높은 상품들을 추천한다.
온라인 쇼핑몰의 활성화로 소비자들의 소비 활동이 활발해짐에 따라 기업들은 매출 증대를 위해 소비자의 상품 트렌드 분석을 수행하고 있다. 기존의 상품 트렌드 분석 기법들은 온라인 쇼핑몰 사용자의 활동만을 고려하여 분석하기 때문에 구매 이력이 없거나 새로운 상품에 대한 트렌드를 파악하기 어렵다. 본 논문에서는 쇼핑몰에서 사용자의 트렌드와 잠재적 고객의 트렌드를 분석하기 위해 온라인 쇼핑몰 데이터와 소셜 네트워크 데이터를 결합한 트렌드 분석 기법을 제안한다. 제안하는 기법은 서비스 내 데이터 분석을 위해 사용자의 활동로그를 분석하고 활동 로그가 없는 잠재적인 사용자들의 관심도를 반영하기 위해 소셜 네트워크 데이터에서 단어 집합 추출을 통해 생성한 핫 토픽을 결합하였다. 최종적으로 상품 지수와 소셜 네트워크에서의 언급수를 활용하여 시간에 따른 상품 트렌드 변화를 탐지한다. 소셜 네트워크 데이터를 활용한 성능 평가를 통해 제안하는 기법의 우수성을 입증한다.
['온라인 쇼핑몰의 대중화로 인해 소비자는 폭 넓은 소비의 기회를 제공받고 있다. 소비자들은 온라인 쇼핑몰에서 제공되는 실제 상품을 

3.9892452004251093